# Assignment

Deadline: 24 novembre ore 23:59.
Solve efficiently a generic n^2-1 puzzle using path search algorithm.

Cost=  total number of actions you need to __evaluate__. An action is something that bring me to a new state. For example the number of swaps to do.

The result is the sequence of action that took you at the end. The goal is not to find a state but a sequence of actions from srtarting point to end point: we do not look for a soluzion but for a sequence of actions.

# Import and Inizialization

In [79]:
from collections import namedtuple
from random import choice
from tqdm.auto import tqdm
import numpy as np
from heapq import heappop, heappush
from typing import Tuple, Union

In [80]:
PUZZLE_DIM = 7
action = namedtuple('Action', ['pos1', 'pos2'])

In [81]:
def available_actions(state: np.ndarray) -> list['Action']:
    x, y = [int(_[0]) for _ in np.where(state == 0)]
    actions = list()
    if x > 0:
        actions.append(action((x, y), (x - 1, y)))
    if x < PUZZLE_DIM - 1:
        actions.append(action((x, y), (x + 1, y)))
    if y > 0:
        actions.append(action((x, y), (x, y - 1)))
    if y < PUZZLE_DIM - 1:
        actions.append(action((x, y), (x, y + 1)))
    return actions



def do_action(state: np.ndarray, action: 'Action') -> np.ndarray:
    new_state = state.copy()
    new_state[action.pos1], new_state[action.pos2] = new_state[action.pos2], new_state[action.pos1]
    return new_state

Function to evaluate the quality of a solution (list of actions) as the total number of actions needed.

In [82]:
def qualily(actions):
    return len(actions)

The state is a numpy array.

We created a function that returns the number of actions from a state pos1 to a state pos2.

Compute 100_000 random actions:

In [83]:
RANDOMIZE_STEPS = 100_000
state = np.array([i for i in range(1, PUZZLE_DIM**2)] + [0]).reshape((PUZZLE_DIM, PUZZLE_DIM))
for r in tqdm(range(RANDOMIZE_STEPS), desc='Randomizing'):
    state = do_action(state, choice(available_actions(state)))
state

Randomizing: 100%|██████████| 100000/100000 [00:01<00:00, 62482.08it/s]


array([[ 9, 44, 18, 12, 40, 39, 38],
       [ 1, 15,  8,  2,  4,  0, 10],
       [30, 46, 48, 16, 23, 27, 17],
       [26, 47, 33, 20, 45, 32, 34],
       [42, 22, 25, 35, 29, 24, 41],
       [43, 21, 28, 31, 14, 19, 13],
       [11, 37,  3,  6,  5,  7, 36]])

Define a function that indicates if we end the search.

In [84]:
def test_goal(solution):
    arr_solution = np.reshape(solution, PUZZLE_DIM*PUZZLE_DIM)
    arr_solution_no_zero = arr_solution[0: len(arr_solution)-1]
    if np.all(arr_solution_no_zero[:-1] <= arr_solution_no_zero[1:]) and arr_solution[len(arr_solution)-1]==0:
        return True
    return False

# A*

We may also want to try to convert the numpy ndarray into a bytes object directly:

In [85]:
def state_to_bytes(state: np.ndarray) -> bytes:
    return state.tobytes()

As heuristic functions we can now try to use 3 different heuristics: 
1. manhattan distance
2. linear conflict
3. walking distance

In [86]:
class PuzzleHeuristicService:
    def __init__(self, goal_state: np.ndarray):
        self.goal_state = goal_state

    def heuristic_manhattan_distance(self, position):
        distance = 0
        size = len(position)
        for i in range(size):
            for j in range(size):
                tile = position[i][j]
                if tile != 0:
                    target_row = (tile - 1) // size
                    target_col = (tile - 1) % size
                    distance += abs(i - target_row) + abs(j - target_col)
        return distance
    

    def heuristic_linear_conflict(self, position):
        conflict = 0
        size = len(position)

        # Row conflicts
        for row in range(size):
            max_val = -1
            for col in range(size):
                value = position[row][col]
                if value != 0 and (value - 1) // size == row:
                    if value > max_val:
                        max_val = value
                    else:
                        conflict += 2

        # Column conflicts
        for col in range(size):
            max_val = -1
            for row in range(size):
                value = position[row][col]
                if value != 0 and (value - 1) % size == col:
                    if value > max_val:
                        max_val = value
                    else:
                        conflict += 2

        return conflict

    def heuristic_walking_distance(self, position):
        # Create a grid to store the walking distances
        size = len(position)
        distance_grid = [[0] * size for _ in range(size)]

        for row in range(size):
            for col in range(size):
                value = position[row][col]
                if value != 0:
                    target_row = (value - 1) // size
                    target_col = (value - 1) % size
                    distance_grid[row][col] = abs(row - target_row) + abs(col - target_col)

        # Calculate the walking distance
        walking_distance = 0
        for row in range(size):
            for col in range(size):
                walking_distance += distance_grid[row][col]

        return walking_distance

    def combined_heuristic(self, state: np.ndarray) -> int:
        if PUZZLE_DIM<=3:
            return self.heuristic_manhattan_distance(state)
        if PUZZLE_DIM<=5:
            return 1*(self.heuristic_manhattan_distance(state) + self.heuristic_linear_conflict(state) + self.heuristic_walking_distance(state))
        return 5*(self.heuristic_manhattan_distance(state) + self.heuristic_linear_conflict(state) + self.heuristic_walking_distance(state))


In [87]:
def enhanced_a_star(initial_state: np.ndarray, final_state: np.ndarray) -> Tuple[Union[list, None], float]:
    """
    Enhanced A* algorithm for the n-puzzle problem using modular heuristics.
    """
    heuristic_service = PuzzleHeuristicService(final_state)

    def calculate_heuristic(state: np.ndarray) -> int:
        return heuristic_service.combined_heuristic(state)

    # Priority queue: (f_score, g_score, current_state, path)
    open_set = []
    heappush(open_set, (calculate_heuristic(initial_state), 0, initial_state.tobytes(), []))
    visited = set()
    optimum = state_to_bytes(final_state)

    cost = 0

    while open_set:
        # Extract node with the lowest f score (f score= cost)
        f_score, g_score, current_bytes, path = heappop(open_set)
        current_state = np.frombuffer(current_bytes, dtype=initial_state.dtype).reshape(initial_state.shape)
        current_score = state_to_bytes(current_state)

        # Check if we finished already:
        if current_score == optimum:
            return path, float(cost) 

        # Add current node to visited
        visited.add(current_score)

        # Generate possible moves:
        for act in available_actions(current_state):
            next_state = do_action(current_state, act)
            next_score = state_to_bytes(next_state)
            if next_score in visited:
                continue

            cost += 1

            # update scores:
            new_g_score = g_score + 1
            new_f_score = new_g_score + calculate_heuristic(next_state)

            # Add new state to openset
            heappush(open_set, (new_f_score, new_g_score, next_state.tobytes(), path + [act]))

    return None, float('inf')  # No solution found


# Main

In [88]:
test_goal = np.arange(1, PUZZLE_DIM*PUZZLE_DIM, 1)
test_goal = np.append(test_goal, 0)
test_goal = test_goal.reshape((PUZZLE_DIM, PUZZLE_DIM))

In [89]:
actions, costValue = enhanced_a_star(state, test_goal)

In [90]:
qualily(actions)

620

In [91]:
costValue

70818.0

In [92]:
state

array([[ 9, 44, 18, 12, 40, 39, 38],
       [ 1, 15,  8,  2,  4,  0, 10],
       [30, 46, 48, 16, 23, 27, 17],
       [26, 47, 33, 20, 45, 32, 34],
       [42, 22, 25, 35, 29, 24, 41],
       [43, 21, 28, 31, 14, 19, 13],
       [11, 37,  3,  6,  5,  7, 36]])

In [93]:
test_goal

array([[ 1,  2,  3,  4,  5,  6,  7],
       [ 8,  9, 10, 11, 12, 13, 14],
       [15, 16, 17, 18, 19, 20, 21],
       [22, 23, 24, 25, 26, 27, 28],
       [29, 30, 31, 32, 33, 34, 35],
       [36, 37, 38, 39, 40, 41, 42],
       [43, 44, 45, 46, 47, 48,  0]])